In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta

In [28]:
CNMSshvol_df = pd.read_csv('F/20210212.txt', sep='|', header=0)
CNMSshvol_df['Date'] =  pd.to_datetime(CNMSshvol_df['Date'], errors='coerce', format='%Y%m%d')

In [29]:
CNMSshvol_df

,Date,Symbol,ShortVolume,ShortExemptVolume,TotalVolume,Market
0,2021-02-12,A,98624.0,6.0,304445.0,"B,Q,N"
1,2021-02-12,AA,414786.0,1121.0,1068127.0,"B,Q,N"
2,2021-02-12,AAA,114.0,0.0,116.0,"Q,N"
3,2021-02-12,AAAU,21799.0,0.0,114520.0,"Q,N"
4,2021-02-12,AAC/U,6904.0,0.0,336370.0,"Q,N"
...,...,...,...,...,...,...
9482,2021-02-12,ZWRKU,8714.0,0.0,101032.0,"B,Q,N"
9483,2021-02-12,ZYME,37226.0,0.0,116882.0,"B,Q,N"
9484,2021-02-12,ZYNE,2289583.0,154854.0,7798350.0,"B,Q,N"
9485,2021-02-12,ZYXI,38630.0,1160.0,71296.0,"B,Q,N"


In [95]:
def calculate_change(start, end):
  return round(((end - start)/start)*100, 2)

#this needs to be converted to an array (should be okay because yfticker takes arrays as an arg)
ticker = 'GME'

open_price = yf.Ticker(ticker).history(period='max')['Open']
close_price = yf.Ticker(ticker).history(period='max')['Close']
df = calculate_change(open_price,close_price)
dataframe=pd.DataFrame(df)
dataframe['Ticker']=ticker

dataframe

,0,Ticker
Date,,
2002-02-13,4.42,GME
2002-02-14,-1.72,GME
2002-02-15,-0.50,GME
2002-02-19,-3.54,GME
2002-02-20,2.86,GME
...,...,...
2021-02-08,-17.14,GME
2021-02-09,-11.13,GME
2021-02-10,0.85,GME


In [96]:
today = datetime.today()
#This date is hardcoded but needs to be a variable as well
start_date = today - timedelta(days=15)
filename = start_date.strftime("%Y%m%d")
finra = pd.DataFrame()
bx = pd.DataFrame()
psx = pd.DataFrame()

days_to_scrape = start_date
while (days_to_scrape < today):
    filename = days_to_scrape.strftime("%Y%m%d")
    try:
        file = pd.read_csv('F/'+filename+'.txt', sep='|', header=0)
        finra = pd.concat([finra, file])
        file = pd.read_csv('B/'+filename+'.txt', sep='|', header=0)
        bx = pd.concat([bx, file])
        file = pd.read_csv('P/'+filename+'.txt', sep='|', header=0)
        psx = pd.concat([psx, file])
        
        print('got data for '+ filename)
    except:
        print('no data for ' + filename)


    days_to_scrape += timedelta(days=1)
    

got data for 20210129
no data for 20210130
no data for 20210131
got data for 20210201
got data for 20210202
got data for 20210203
got data for 20210204
got data for 20210205
no data for 20210206
no data for 20210207
got data for 20210208
got data for 20210209
got data for 20210210
got data for 20210211
got data for 20210212


# Calculate shortvol/totalvol for all markets

In [97]:
# TODO: Make faster by finding tickers first
import warnings
warnings.filterwarnings('ignore')

finra_ticker = finra[finra['Symbol'] == ticker]
bx_ticker = bx[bx['SYMBOL'] == ticker]
psx_ticker = psx[psx['SYMBOL'] == ticker]

for i, row in finra_ticker.iterrows():
    finra_ticker.at[i,'volume_percent'] = row['ShortVolume']/row['TotalVolume']
    
for i, row in bx_ticker.iterrows():
    bx_ticker.at[i,'volume_percent'] =  row['SHORT VOLUME']/row['TOTAL VOLUME']
    
for i, row in psx_ticker.iterrows():
    psx_ticker.at[i,'volume_percent'] =  row['SHORT VOLUME']/row['TOTAL VOLUME']

In [98]:
finra_ticker

,Date,Symbol,ShortVolume,ShortExemptVolume,TotalVolume,Market,volume_percent
3630,20210129,GME,8814229.0,527920.0,16327706.0,"B,Q,N",0.539833
3623,20210201,GME,6982444.0,364890.0,12820226.0,"B,Q,N",0.503354
3651,20210202,GME,16358136.0,1073011.0,29733410.0,"B,Q,N",0.550160
3631,20210203,GME,9638240.0,532939.0,17735739.0,"B,Q,N",0.543436
3623,20210204,GME,14272780.0,869183.0,28355377.0,"B,Q,N",0.503354
3635,20210205,GME,19063724.0,1106467.0,34566363.0,"B,Q,N",0.551511
3689,20210208,GME,6404809.0,82425.0,11348581.0,"B,Q,N",0.564371
3682,20210209,GME,6516142.0,277522.0,12057907.0,"B,Q,N",0.540404
3673,20210210,GME,6587323.0,241069.0,18257999.0,"B,Q,N",0.360791
3678,20210211,GME,2234229.0,33414.0,6204532.0,"B,Q,N",0.360096


In [99]:
bx_ticker

,DATE,SYMBOL,SHORT VOLUME,TOTAL VOLUME,MARKET,volume_percent
2450,20210129,GME,44729,56858,B,0.786679
2415,20210201,GME,95281,115240,B,0.826805
2365,20210202,GME,331287,443240,B,0.747421
2383,20210203,GME,88922,132993,B,0.668622
2366,20210204,GME,164438,236358,B,0.695716
2420,20210205,GME,345068,490824,B,0.703038
2461,20210208,GME,43273,57312,B,0.597491
2470,20210209,GME,39249,57542,B,0.682093
2461,20210210,GME,68736,115041,B,0.597491
2466,20210211,GME,23797,44186,B,0.538564


In [100]:
psx_ticker

,DATE,SYMBOL,SHORT VOLUME,TOTAL VOLUME,MARKET,volume_percent
2030,20210129,GME,19978,76886,X,0.259839
2004,20210201,GME,36544,60620,X,0.602837
1966,20210202,GME,22974,46999,X,0.733386
1957,20210203,GME,14776,32943,X,0.448532
1964,20210204,GME,28599,54363,X,0.526075
2000,20210205,GME,161692,319081,X,0.506743
2006,20210208,GME,30585,37475,X,0.816144
2039,20210209,GME,48585,83443,X,0.582254
2083,20210210,GME,26003,47000,X,0.553255
2082,20210211,GME,22930,41846,X,0.547962


In [101]:
psx_ticker=psx_ticker.rename(columns={"SYMBOL": "symbol", 'DATE':'date'})
bx_ticker=bx_ticker.rename(columns={"SYMBOL": "symbol",'DATE':'date'})
finra_ticker=finra_ticker.rename(columns={"Symbol": "symbol",'Date':'date'})

In [102]:
final_df =  pd.merge(pd.merge(psx_ticker, bx_ticker, on=['symbol','date']),finra_ticker, on=['symbol','date'])

In [103]:
final_df

,date,symbol,SHORT VOLUME_x,TOTAL VOLUME_x,MARKET_x,volume_percent_x,SHORT VOLUME_y,TOTAL VOLUME_y,MARKET_y,volume_percent_y,ShortVolume,ShortExemptVolume,TotalVolume,Market,volume_percent
0,20210129,GME,19978,76886,X,0.259839,44729,56858,B,0.786679,8814229.0,527920.0,16327706.0,"B,Q,N",0.539833
1,20210201,GME,36544,60620,X,0.602837,95281,115240,B,0.826805,6982444.0,364890.0,12820226.0,"B,Q,N",0.503354
2,20210202,GME,22974,46999,X,0.733386,331287,443240,B,0.747421,16358136.0,1073011.0,29733410.0,"B,Q,N",0.550160
3,20210203,GME,14776,32943,X,0.448532,88922,132993,B,0.668622,9638240.0,532939.0,17735739.0,"B,Q,N",0.543436
4,20210204,GME,28599,54363,X,0.526075,164438,236358,B,0.695716,14272780.0,869183.0,28355377.0,"B,Q,N",0.503354
5,20210205,GME,161692,319081,X,0.506743,345068,490824,B,0.703038,19063724.0,1106467.0,34566363.0,"B,Q,N",0.551511
6,20210208,GME,30585,37475,X,0.816144,43273,57312,B,0.597491,6404809.0,82425.0,11348581.0,"B,Q,N",0.564371
7,20210209,GME,48585,83443,X,0.582254,39249,57542,B,0.682093,6516142.0,277522.0,12057907.0,"B,Q,N",0.540404
8,20210210,GME,26003,47000,X,0.553255,68736,115041,B,0.597491,6587323.0,241069.0,18257999.0,"B,Q,N",0.360791
9,20210211,GME,22930,41846,X,0.547962,23797,44186,B,0.538564,2234229.0,33414.0,6204532.0,"B,Q,N",0.360096
